In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [2]:
from case.code_generator import get_generator
from utils.notebook_printer import NotebookPrinter

generator = get_generator()
printer = NotebookPrinter(generator)

In [3]:
question = "Show me the code for swapping 10 usdt to dai using 1inch"
await printer.ask_streaming(question)

Here is the code for swapping 10 USDT to DAI using the 1inch protocol:

```javascript
import { formatUnits, parseUnits } from 'viem'
import { swapTx } from '../prebuilt-tx/1inch'

// Constants
const chainId = 1; // Mainnet
const userAddress = 'YOUR_WALLET_ADDRESS'; // Replace with your wallet address
const usdtAddress = '0xdac17f958d2ee523a2206206994597c13d831ec7'; // USDT contract address
const daiAddress = '0x6b175474e89094c44da98b954eedeac495271d0f'; // DAI contract address
const usdtAmount = parseUnits('10', 6); // 10 USDT (6 decimals)

// Execute the swap
const executeSwap = async () => {
  try {
    const swapResult = await swapTx({
      chainId: chainId,
      userAddress: userAddress,
      srcTokenAddress: usdtAddress,
      srcTokenSymbol: 'USDT',
      srcTokenDecimals: 6,
      srcAmount: usdtAmount,
      dstTokenAddress: daiAddress,
      dstTokenSymbol: 'DAI',
      dstTokenDecimals: 18,
    });

    console.log(`Transaction: ${JSON.stringify(swapResult.tx)}`);
    console.log(`Amount of DAI received: ${formatUnits(swapResult.dstAmount, 18)}`);
  } catch (error) {
    console.error('Error executing swap:', error);
  }
};

// Call the function to execute the swap
executeSwap();
```

### Explanation:
- **Imports**: The necessary functions and constants are imported.
- **Constants**: The chain ID, user wallet address, and token addresses for USDT and DAI are defined.
- **Amount**: The amount of USDT to swap (10 USDT) is parsed into the appropriate format.
- **Swap Execution**: The `swapTx` function is called with the necessary parameters to perform the swap. The result is logged to the console, showing the transaction details and the amount of DAI received.

Make sure to replace `'YOUR_WALLET_ADDRESS'` with your actual wallet address before running the code.

/Users/clement/Developer/ai/tx-generator/.venv/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


Using tool: case_code_retriever with inputs: {'query': 'swap 10 usdt to dai using 1inch'}
--
Using retriever
--
Retrieved documents: [{'case': 'prebuilt-tx', 'file': '1inch.ts'}, {'case': 'zircuit-ethena', 'file': 'usdt-case.ts'}, {'case': 'prebuilt-tx', 'file': 'dinero.ts'}, {'case': 'prebuilt-tx', 'file': 'merlinswap.ts'}]
--
Done tool: case_code_retriever
--


In [4]:
question = "Show me the code for swapping 10 usdt to dai"
await printer.ask(question)

Here is the code for swapping 10 USDT to DAI using a decentralized exchange (like Uniswap):

```javascript
import { encodeFunctionData, formatUnits, getContract, parseUnits } from 'viem'
import { mainnet } from 'viem/chains'
import {
  Context,
  InputType,
  PartialBatchCase,
  Tx,
  WalletType,
} from '@/models/cases/v3/types'
import PublicClient from '@services/publicClient'
import { TokenAddress } from '../tokenAddresses'
import UniswapQuoterV3 from './abi/UniswapQuoterV3.json'
import UniswapRouterV3 from './abi/UniswapRouterV3.json'
import USDT from './abi/USDT.json'
import DAI from './abi/DAI.json'

const usdtAddr = TokenAddress.Mainnet.USDT
const daiAddr = TokenAddress.Mainnet.DAI
const uniswapQuoterAddr = '0xb27308f9F90D607463bb33eA1BeBb41C27CE5AB6'
const uniswapRouterAddr = '0xE592427A0AEce92De3Edee1F18E0157C05861564'

const swapUSDTToDAI: PartialBatchCase = {
  id: 'swap_usdt_to_dai',
  attributes: [
    { id: 'protocols', name: 'Protocols' },
    { id: 'txn_count', name: 'TXN count', value: '2' },
  ],
  networkId: mainnet.id,
  atomic: true,
  supportedWalletTypes: [WalletType.AA, WalletType.EOA],
  inputs: [
    {
      name: 'USDT Amount',
      inputType: InputType.ERC20Amount,
      description: 'Amount of USDT to swap',
      validate: decimalValidator(6, BigInt(0)),
      options: {
        token: TokenAddress.Mainnet.USDT,
      },
      symbol: 'USDT',
    },
  ],
  render: async (context: Context) => {
    const inputs = context.inputs as string[]
    const txs: Tx[] = []

    const client = PublicClient.get(context.chain)

    const usdt = getContract({
      address: usdtAddr,
      abi: USDT,
      client: client,
    })

    const dai = getContract({
      address: daiAddr,
      abi: DAI,
      client: client,
    })

    const uniswapQuoter = getContract({
      address: uniswapQuoterAddr,
      abi: UniswapQuoterV3,
      client: client,
    })

    const uniswapRouter = getContract({
      address: uniswapRouterAddr,
      abi: UniswapRouterV3,
      client: client,
    })

    const usdtAmount = parseUnits('10', 6) // Swapping 10 USDT

    const daiAmount = (await uniswapQuoter.read.quoteExactInputSingle([
      usdtAddr, // token in
      daiAddr, // token out
      '100', // fee
      usdtAmount, // amount in
      '0', // sqrtPriceLimitX96
    ])) as bigint

    const daiFormatAmount = formatUnits(daiAmount, 18)

    txs.push({
      name: `Swap`,
      description: `Swapping 10 USDT to ${daiFormatAmount} DAI`,
      to: uniswapRouter.address,
      value: 0n,
      data: encodeFunctionData({
        abi: uniswapRouter.abi,
        functionName: 'exactInputSingle',
        args: [
          [
            usdtAddr, // token in
            daiAddr, // token out
            '100', // fee
            context.account.address, // recipient
            Math.floor(Date.now() / 1000) + 300, // deadline
            usdtAmount, // amountIn
            daiAmount, // amountOutMinimum
            '0', // sqrtPriceLimitX96
          ],
        ],
      }),
      abi: uniswapRouter.abi,
    })

    return txs
  },
}

export default swapUSDTToDAI
```

### Explanation:
- This code defines a function to swap 10 USDT to DAI using the Uniswap protocol.
- It retrieves the necessary contracts for USDT, DAI, and the Uniswap router.
- It calculates the amount of DAI that can be received for 10 USDT using the Uniswap quoter.
- Finally, it constructs a transaction to perform the swap on the Uniswap router.

In [5]:
question = "Show me the code for supplying 10 usdt to compound"
await printer.ask(question)

Here is the code for supplying 10 USDT to Compound:

```javascript
import { encodeFunctionData, formatUnits, getContract, parseUnits } from 'viem'
import { mainnet } from 'viem/chains'
import {
  Context,
  InputType,
  PartialBatchCase,
  Tx,
  WalletType,
} from '@/models/cases/v3/types'
import PublicClient from '@services/publicClient'
import { TokenAddress } from '../tokenAddresses'
import CompoundComet from './abi/CompoundComet.json'
import USDT from './abi/USDT.json'
import type { Abi } from 'abitype'

const usdtAddr = TokenAddress.Mainnet.USDT
const compoundCometAddr = '0xYourCompoundCometAddress' // Replace with actual Compound Comet address

const usdtCase: PartialBatchCase = {
  id: 'supply_usdt_to_compound',
  attributes: [
    { id: 'protocols', name: 'Protocols' },
    { id: 'txn_count', name: 'TXN count', value: '1' },
  ],
  networkId: mainnet.id,
  atomic: true,
  supportedWalletTypes: [WalletType.AA, WalletType.EOA],
  inputs: [
    {
      name: 'USDT Amount',
      inputType: InputType.ERC20Amount,
      description: 'Amount of USDT to supply to Compound',
      validate: (value) => value > 0 && value <= 10, // Validate for 10 USDT
      options: {
        token: usdtAddr,
      },
      symbol: 'USDT',
    },
  ],
  render: async (context: Context) => {
    const inputs = context.inputs as string[]
    const txs: Tx[] = []

    const client = PublicClient.get(context.chain)

    const usdt = getContract({
      address: usdtAddr,
      abi: USDT,
      client: client,
    })

    const usdtAmount = parseUnits(inputs[0], 6) // Assuming USDT has 6 decimals

    // Approve USDT for Compound
    const usdtAllowance = (await usdt.read.allowance([
      context.account.address,
      compoundCometAddr,
    ])) as bigint

    if (usdtAmount > usdtAllowance) {
      txs.push({
        name: `Approve`,
        description: `Approve ${inputs[0]} USDT to Compound`,
        to: usdt.address,
        value: 0n,
        data: encodeFunctionData({
          abi: usdt.abi,
          functionName: 'approve',
          args: [compoundCometAddr, usdtAmount],
        }),
        abi: usdt.abi as Abi,
      })
    }

    // Supply USDT to Compound
    txs.push({
      name: `Supply`,
      description: `${formatUnits(usdtAmount, 6)} USDT to Compound`,
      to: compoundCometAddr,
      value: 0n,
      data: encodeFunctionData({
        abi: CompoundComet,
        functionName: 'supply',
        args: [usdtAddr, usdtAmount],
      }),
      abi: CompoundComet as Abi,
    })

    return txs
  },
}

export default usdtCase
```

### Explanation:
1. **Imports**: The necessary libraries and contracts are imported.
2. **Constants**: The USDT address and the Compound Comet address are defined.
3. **PartialBatchCase**: A case is created to handle the supply of USDT to Compound.
4. **Input Validation**: The input for the USDT amount is validated to ensure it is between 0 and 10.
5. **Transaction Creation**:
   - An approval transaction is created if the allowance is insufficient.
   - A supply transaction is created to send the specified amount of USDT to Compound.

Make sure to replace `'0xYourCompoundCometAddress'` with the actual address of the Compound Comet contract you are using.